# Contact Invariant Optimization

This notebook gives an overview of Contact Invariant Optimization by Mordatch et al.(here: <https://homes.cs.washington.edu/~todorov/papers/MordatchSCA12.pdf>). Specifically the implementation involving manipulation.

Solving for manipulation trajectories involving contact is a notoriously difficult optimization problem due to discontinuities introduced by the hybrid space. CIO attempts to solve this issue by smoothing the transitions between contact dynamics and free space dynamics.

Given a problem formulation consisting of desired constraints on the resulting motion, CIO attempts to solve for the necessary places to make contact, the forces to apply at these contact points, and the trajectory of the manipulated object and fingers all with random initialization of the decision variables. The paper describes all of the constraints used in the final optimization (L-BFGS). Here is a video of the kinds of manipulation plans CIO claims to be able to find.

In [4]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('Gzt2UoxYfAQ')

Today I will given an overview of the method and an API I've developed for using CIO (spoiler alert: it doesn't work). Then I will give an a example and we can all play around with it.

# Method and Implementation

My implementation makes some similifications from the method discussed in the paper. Given an initial state $s_0$ and a goal, CIO attempts to solve for the state trajectory $s(t)$ for t $\in [0,T]$.

## State

The state of the hand is represented as the pose and twist (linear and angular velocity) of two grippers, $s_H = [x_{G1}, \dot{x}_{G1}, x_{G1}, \dot{x}_{G1}]$. 

There is a single manipulated object with $N_{contacts}=3$ potential contact points: 1 from the ground and 2 from the grippers.

The state of the manipulated object is the pose and twist of the object, $[x_O, \dot{x}_O]$. For each contact, $j \in \{1,...,N_{contacts}\}$ the state also includes the force direciton of the contact, $f_j$, the origin point for the contact in the object's local frame, $r^O_j$, and an auxiliary variable, $c_j$, which specifies if a contact is active. $c \in [0,1]$ is the only bound decision variable. $c=1$ indicates a contact, and $c=0$ indicates no contact. The fact that it is a *continuous* variable is the key to CIO and finding solutions.

The entire system state is therefore:
$ s = [x_{G1}, \dot{x}_{G1}, x_{G1}, \dot{x}_{G1}, x_O, \dot{x}_O, f_0, r^O_0, c_0, f_1, r^O_1, c_1, f_2, r^O_2, c_2]$

CIO aims to solve for the entire state for $N_{keyframes}$ of the task. Different interpolation methods are used to calculate the state, $s(t)$ for any $t \in [0,T]$ from the keyframes. The poses and twists use cubic spline interpolation, the forces and distances use linear interpolation, and the contact variables use constant interpolation making them piece-wise constant. Therefore the entire system state which CIO aims to solve is $S= [s_k} for k \in \{0,...,N_{keyframes}}$.

![CIO State](state.png)

First we initialize the list of objects in the world which can be of type Line, Circle, or Rectangle. We then define the initial state. Each object is defined as follows:

Object(2Dpos, angle, twist, actuated, pose_index, contact_index, step_size)
2dpos (list of floats length 2]
angle float in rad
twist list of 3 floats
actuated True is object can exert force
pose_index int
contact_index int
step_size float (for collision checking)

In [6]:
import numpy as np
from world import Line, Rectangle, Circle

# ground: origin is left
ground = Line((0.0, 0.0), 0.0, 30.0, contact_index = 2)

# circle: origin is center of the circle
rad = 5.0
box = Circle(pose=(5.0, rad), angle=0.0, radius=rad, vel = (0.0, 0.0, 0.0), pose_index = 2)
# box: origin is left bottom of box
#box = Rectangle((5.0, 0.0), np.pi/2, 10.0, 10.0, vel = (0.0, 0.0, 0.0), pose_index = 2)

# gripper1: origin is bottom of line
gripper1 = Line((0.0, 20.0), np.pi/2, 2.0, pose_index = 0, contact_index = 0,\
                actuated = True)

# gripper2: origin is bottom of line
gripper2 = Line((10.0, 20.0), np.pi/2, 2.0, pose_index = 1, contact_index = 1,\
                actuated = True)

objects = [ground, box, gripper1, gripper2]

There are many variables that need to be initialized.

In [7]:
num_moveable_objects = len(objects) - 1 # don't count the ground

# set params
import params as p
paramClass = p.Params({}, num_moveable_objects)
p.set_global_params(paramClass)

These objects then need to be used to construct the initial decision variables. Here you can adjust the initial contact variables f, r, and c.

In [8]:
# create the initial system state: s0
s0 = np.zeros(p.len_s)

# fill in object poses and velocities
for object in objects:
    if object.pose_index != None:
        s0[6*object.pose_index:6*object.pose_index+2] = object.pose
        s0[6*object.pose_index+2] = object.angle
        s0[6*object.pose_index+3:6*object.pose_index+6] = object.vel

# initial contact information (just in contact with the ground):
# [fxj fyj rOxj rOyj cj for j in N]
# j = 0 gripper 1 contact
# j = 1 gripper 2 contact
# j = 2 ground contact
# rO is in object (box) frame
con0 = [0.0, 0.0,  -5.0, 10.0, 1.0] # gripper1
con1 = [0.0, 0.0, 5.0, 10.0, 1.0] # gripper2
con2 = [0.0, 10.0,  0.0, -5.0, 1.0] # ground

s0[18:p.len_s] = (con0 + con1 + con2)

For now we will initialize the remaining $N_{keyframes}-1$ state decision variables to $s_0$ with some Gaussian noise.

In [9]:
from util import add_noise

# initialize traj to all be the same as the starting state
S0 = np.zeros(p.len_S)
for k in range(p.K):
    S0[k*p.len_s:k*p.len_s+p.len_s] = s0
    
add_noise(S0);

Finally, we define the goal as the desired pose of the manipulated object.

In [10]:
goal = ("box", (50.0, rad, np.pi/2))

At any point you can visualize the state (Thanks Tom!) using the following command.

In [11]:
from CIO import visualize_result

visualize_result(S0, s0, objects, goal, 'initial.gif');

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


The previous cell generates a .gif in the local directory. Play it by **restarting the kernel** and running following cell.

![Initial State](initial.gif)

The black lines represent the grippers, the red circle is the manipulated object, the blue circles are the current contacts ($r^O_j$), and the green dot is the goal. The shading of the circles represent the value of $c_j$. Black lines represent? If f_j should move them to the blue circles.


## Objective Cost Terms

The objective function for the optimization consists of several functions. I will now go into each of them.

### Contact-Invariant Cose (cis)

A contact $j$ is active when $c_j$ is close to 1. When this is true, the corresponding gripper must be constrained to be in contact with the object. This means that the distance between the gripper and the position of the gripper projected onto the surface of the object must be closer to 0. This objective function also constraints the derivative of this position to enfore a no slipping constraint.

### Physics Violation Cost (physics)

The physical constraints account for the fact that forces on the object from the contacts must balance. This term also includes a regularization on the contact forces to keep them as small as possible. It also restricts the contact forces to lie in the friction cone.

### Kinematic Violation (kinematics)

This limits the joint angles, the distance the fingers can extend from the hand, and all collisions between objects. In my implementation this is only collision costs as there are no fingers.

### Task Cost (task)

This is the constraint which defines the task goal. It simply states that we want to minimize the distance between the desired pose and the final pose at the last time step. 

### Smoothness (accels)

This term regularized the accelerations of the object and the gripper to achieve smoother actions.

## Optimization

CIO runs two separate optimization phases. First, the physics and cis costs are downweighted by .1 of their original values. This allows the optimizer to find solutions without worrying to much abotu if it makes sense physically. Then in the second phase is initialized with the solution to the first, and the terms are weighted as follows.

![](weights.png)

Now we are ready to run CIO with the default parameters.

In [5]:
from CIO import CIO
phase_info = CIO(goal, objects, s0, S0)

NameError: name 'goal' is not defined

You can visualize the intermediate and final solutions by **restarting the kernel** and running the following cells.

### Phase 0
![](phase_0.gif)

### Phase 1
![](phase_1.gif)